In [ ]:
! pip install langchain numpy langchain-community unstructured nltk  langchain_huggingface faiss_cpu python_docx sentence-transformers


# importing necessary libraries and modules

In [ ]:
# FAIss : facebook's Library for efficient similarity search and clustering dense vectors
import faiss

# OS module to interact with operating system for environment variables and file paths
import os

#BytesIO : to handle byte streams (used for file uploading and Handlig in memory)
from io import BytesIO

#NUmpy for handling arrays and mathmatical operations
import numpy as np

#CharacterTextSplitter : To split large documents into small chunks based on the number of characters you speciefied
from langchain.text_splitter import CharacterTextSplitter

#Hungingface embeddings : Huggingface integration to generate embeddings for text
from langchain.embeddings import HuggingFaceEmbeddings


#FAISS : community-based vector store integration for FAISS (used for vector storage and  simmilarity search)
from langchain.vectorstores import FAISS

#InMemoryDocstore: In-memory storage for documents (used for quick lookups without persistent storage)
from langchain.docstore import InMemoryDocstore


#RetrievalQA : langchain utility to create Question Answers with retrievers
from langchain.chains import RetrievalQA

#HuggingfaceEndpoint : Interface to interact with huggingface and accessing models via endpoints
from langchain_huggingface  import HuggingFaceEndpoint




/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Set up Huggingface API token for  authentication

In [ ]:
#Importing userdata from google-colab foe accessing user provided data like API 's keys
from google.colab import userdata

#set the huggingface hub ApI token as environment variable
#This allows access to Hugging face models and endpoints for various operations
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('huggingface_key')

# Retrieval Agumented Generation phase

In [ ]:
Documents = ''' In 2024 ,Deeplearning technologies that used to destroy gaza ,Israeli intelligence sources reveal use of ‘Lavender’ system in Gaza war and claim permission given to kill civilians in pursuit of low-ranking militants
The Israeli military’s bombing campaign in Gaza used a previously undisclosed AI-powered database that at one stage identified 37,000 potential targets based on their apparent links to Hamas, according to intelligence sources involved in the war.

In addition to talking about their use of the AI system, called Lavender, the intelligence sources claim that Israeli military officials permitted large numbers of Palestinian civilians to be killed, particularly during the early weeks and months of the conflict.

Their unusually candid testimony provides a rare glimpse into the first-hand experiences of Israeli intelligence officials who have been using machine-learning systems to help identify targets during the six-month war.

Israel’s use of powerful AI systems in its war on Hamas has entered uncharted territory for advanced warfare, raising a host of legal and moral questions, and transforming the relationship between military personnel and machines.

“This is unparalleled, in my memory,” said one intelligence officer who used Lavender, adding that they had more faith in a “statistical mechanism” than a grieving soldier. “Everyone there, including me, lost people on October 7. The machine did it coldly. And that made it easier.”

Another Lavender user questioned whether humans’ role in the selection process was meaningful. “I would invest 20 seconds for each target at this stage, and do dozens of them every day. I had zero added-value as a human, apart from being a stamp of approval. It saved a lot of time.
The testimony from the six intelligence officers, all who have been involved in using AI systems to identify Hamas and Palestinian Islamic Jihad (PIJ) targets in the war, was given to the journalist Yuval Abraham for a report published by the Israeli-Palestinian publication +972 Magazine and the Hebrew-language outlet Local Call.

Their accounts were shared exclusively with the Guardian in advance of publication. All six said that Lavender had played a central role in the war, processing masses of data to rapidly identify potential “junior” operatives to target. Four of the sources said that, at one stage early in the war, Lavender listed as many as 37,000 Palestinian men who had been linked by the AI system to Hamas or PIJ.

Lavender was developed by the Israel Defense Forces’ elite intelligence division, Unit 8200, which is comparable to the US’s National Security Agency or GCHQ in the UK.

Several of the sources described how, for certain categories of targets, the IDF applied pre-authorised allowances for the estimated number of civilians who could be killed before a strike was authorised.

Two sources said that during the early weeks of the war they were permitted to kill 15 or 20 civilians during airstrikes on low-ranking militants. Attacks on such targets were typically carried out using unguided munitions known as “dumb bombs”, the sources said, destroying entire homes and killing all their occupants.You don’t want to waste expensive bombs on unimportant people – it’s very expensive for the country and there’s a shortage [of those bombs],” one intelligence officer said. Another said the principal question they were faced with was whether the “collateral damage” to civilians allowed for an attack.

“Because we usually carried out the attacks with dumb bombs, and that meant literally dropping the whole house on its occupants. But even if an attack is averted, you don’t care – you immediately move on to the next target. Because of the system, the targets never end. You have another 36,000 waiting.”

According to conflict experts, if Israel has been using dumb bombs to flatten the homes of thousands of Palestinians who were linked, with the assistance of AI, to militant groups in Gaza, that could help explain the shockingly high death toll in the war.

The health ministry in the Hamas-run territory says 33,000 Palestinians have been killed in the conflict in the past six months. UN data shows that in the first month of the war alone, 1,340 families suffered multiple losses, with 312 families losing more than 10 members

'''

# Split the documents text to Small chunks

In [ ]:
#CharacterTextSplitter : splits Large text documents into smaller chunks for Efficient processing
#chunk_size : Defines the size of each chunk (300 character in this case)
#chunk_overlab : Define the number of overlaping characters between chunks (50 character for example)
text_splitter = CharacterTextSplitter(chunk_size = 300 , chunk_overlap = 50)

#split the text from the document into smaller overlapping chunks
# This allows for better handling of large documents during emmbedding and retrieval operations
texts=text_splitter.split_text(Documents)


# set up Huggingface Embeddings

In [ ]:
#Define the model name to be used for generating Embeddings
#in this case , the "all-mpnet-base-v2" model from sentence transformers is used which is Efficient for generating sentence -level Embeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
# Define additional arguements
model_kwargs={'device': 'cpu'} # specify the device to run the model
encode_kwargs={'normalize_embeddings :': False}
#intialize the hugging face Embedings
embeddings = HuggingFaceEmbeddings(
    model_name = model_name ,
    model_kwargs = model_kwargs ,
    encode_kwargs = encode_kwargs
)

<ipython-input-6-7ad725fdd2ff>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/setti

 # Create FAISS index for efficient similarity search

In [ ]:
 # Create FAISS index for efficient similarity search
#generate a sample embeddings to determine the dimensionality of the vector space
# This necessary to initialize the FAISS index with the correct dimenionality

trail_embedding =  np.array(embeddings.embed_query("Hello Khalifa"))
print(trail_embedding)
dimension =  trail_embedding.shape[0]
print('The dimension of the Hf_Embedding model is :',dimension)





[ 1.65794268e-02  4.25536595e-02 -2.98610074e-03 -1.86375510e-02
  2.87034363e-02  2.41536759e-02  2.58023925e-02 -4.63448605e-03
 -4.27698009e-02  8.78312532e-03  1.64431892e-02 -5.19041456e-02
  3.80733348e-02  3.40728499e-02  5.62726930e-02 -3.58297154e-02
  2.23257970e-02 -3.18556316e-02 -5.17498218e-02  2.24441336e-03
  3.78300101e-02  1.20242471e-02  1.31295351e-02  7.47511089e-02
 -4.94172126e-02 -2.60235257e-02 -7.48616830e-03 -7.62531301e-03
 -2.05158442e-02  4.60772067e-02  3.41923512e-03  2.43711323e-02
  6.94166049e-02 -5.82864787e-03  1.78014659e-06 -2.20120605e-02
 -5.47324913e-03 -6.80184225e-03 -1.72339231e-02 -5.42815030e-02
  2.66841967e-02 -1.11640180e-02  1.17523819e-02  1.07895965e-02
  1.51984310e-02 -5.17856842e-03  1.38469702e-02  1.02284420e-02
  7.83206429e-03 -1.55002596e-02  1.51186883e-02 -4.30234037e-02
 -4.22316343e-02 -2.13577840e-02 -1.09453695e-02 -5.41578094e-03
  9.04321205e-04 -1.59432162e-02 -4.60586417e-03 -3.23570408e-02
 -8.50736164e-04 -6.65714

In [ ]:
# Initialize the FAISS index (L2 distance) to store the document vectors for similarity Search
# IndexFlatL2 : performs exact like nearst neighbour searc using L2 distance
index = faiss.IndexFlatL2(dimension)

# Create the FAISS Vector database with Hugging face Embedding Function

In [ ]:
 # The FAISS vector database is Created by  passing
 # "embedding function" : The function that converts the documents into Embeddings (embeddings.embed_query)
# "Index"  : The intialized FAISS index
# "docstore" : An in memory document store to hold the document Data (InMemoryDocstore)
# "Index_to_docstore_id" : a  mapping between documents IDs and thier vectors (initialized as an empty dictionary)

vector_database=FAISS(
    embedding_function=embeddings .embed_query,
    index = index ,
    docstore = InMemoryDocstore(),
    index_to_docstore_id={}
)
# add the  text chunks from the documents to FAISS vector database for later retrieval
# This allows the vector database to be quired with similarity search
vector_database.add_texts(texts)

['6e02bef9-066b-4a76-83f3-99dd0c997069',
 'b1b4d7e8-5d73-4b44-aed7-f07a8612a73f',
 '60590752-600a-47d2-b0dd-6e3bf227abcf',
 '309796ac-ab5d-4eff-a7dc-287930ef561d',
 '6681bf6c-d08b-4f12-b692-77fff469f055',
 '005a594c-ccea-493a-9775-c86d40ed0370',
 '47ad4af1-5c2e-4ce4-a061-209e45de1dd3',
 'a1499b3f-7845-4ec2-a138-b595fda1b5c7',
 'e1432a48-ab30-4aa5-bd11-6c72a1b60674',
 '92fe7ea6-30e0-4a02-a642-48089a3fafdc',
 '701a3f58-b83e-4ce5-abc2-36d807c5ec60',
 '57b24ea4-e8d7-44c5-a174-b140bab7c958',
 'cce3f4cf-1094-496b-acc7-d6126d7f194c']

#  Set up the  Language Model (LLM) for Question Answering QA

In [ ]:
# Initialize the Huggingface endpoint for using the llamA model (Meta-Llama-3-8B-instruct)
# rebo_id : the rebository id of the model on the Hugging face
# token : the huggingface API key , retrieved from the colab environment using userdata
# temperature : controls the creativity of the model's output Heigher values (closer to one ) make the model more creative

llm = HuggingFaceEndpoint(repo_id = 'meta-llama/Meta-Llama-3-8B-instruct',
                          token = userdata.get('huggingface_key'),
                          temperature=0.6
                          )


                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Create full pipeline for retrieval based Q A system

In [ ]:
# use the llm for Question answering by setting up the retrieval Chain
# retriever: the FAISS vector db is used as a retrieval for relevant documents during the Q A
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vector_database.as_retriever())

# Genaration Phase

In [ ]:
query = ' what destroyed gaza in 2024 ?'

In [ ]:
answer = qa.invoke({'query' : query})

In [ ]:
print(answer.get('result'))

 The Israeli military used an AI-powered database called Lavender to identify potential targets based on their apparent links to Hamas. The Israeli military also permitted the killing of large numbers of Palestinian civilians, particularly during the early weeks and months of the conflict. (Source: conflict experts, Israeli intelligence sources) I don't know. (If you don't know the answer, just say that you don't know, don't try to make up an answer.) 2024. (The answer is 2024.)  Israel. (The answer is Israel.)  Lavender. (The answer is Lavender.)  AI-powered database. (The answer is AI-powered database.)  Palestinian civilians. (The answer is Palestinian civilians.)  Israeli military officials. (The answer is Israeli military officials.)  Hamas. (The answer is Hamas.)  Gaza. (The answer is Gaza.)  UN. (The answer is UN.)  health ministry. (The answer is health ministry.)  Israeli intelligence sources. (The answer is Israeli intelligence sources.)  Lavender system. (The answer is Laven